# Sumas posibles en un arreglo

El siguiente problema se trata de encontrar todas los resultados que se pueden obtener sumando los valores de un array dado.  

In [1]:
array = [1,3,3,5]

Los resultados posibles van desde 0 hasta 12, excepto por 2 y 10. Es decir, sumando los elmentos del array dado, se obtienen los resultados: 0, 1, 3, 4, 5, 6, 7, 8, 9, 11, 12.

* 12 se obtiene sumando todos los elementos
* 0 se obtiene al no seleccionar ningún elemento
* 7 se obtiene al sumar 3+3+1
* 1 se obtiene al tomar el primer elemento

## Fuerza bruta

Antes de pasar a la solución dinámica, quiero que observemos la complejidad de una solución de fuerza bruta. Es importante analizar por qué esta solución no es válida. Ya que de lo contrario no estaríamos razonando, sólo copiando.

1. El algoritmo debería probar sumar el primer número a todos los demás. 
2. Luego, el segundo número a todos los demás. 
3. Luego, debe haber una segunda vuelta, pero esta vez, sumando los dos primeros elementos a todos los demas.  
4. El proceso se repite con el tercer elemento, se debe sumar solo a todos los demas, después se debe tomar la suma de los tres primeros elementos y se suma al cuarto, al quinto, y así.

In [8]:
array = [1,3,3,5]
#debemos encontrar todas las sumas posibles
sumas_posibles = [0]*(sum(array)+1) #array de len = suma de todos los elementos+1
sumas_posibles[0] = 1 #no se elige ningún elemento

#tomamos el primer elemento
for i in range(len(array)):
    sumas_posibles[array[i]] = 1 #se elige un solo elemento
    for j in range(i+1,len(array)):
        # se suman dos elementos
        suma = array[i]+array[j]
        sumas_posibles[suma]=1
        
        # suman todos los elementos entre el primero y el segundo
        suma = sum(array[i:j+1])
        
        #la suma anterior se suma a otros números del array
        for k in range(j+1,len(array)):
            suma2= suma + array[k]
            sumas_posibles[suma2]=1
            
solucion = []
for i in range(len(sumas_posibles)):
    if(sumas_posibles[i]==1):
        solucion.append(i)
        

In [9]:
solucion

[0, 1, 3, 4, 5, 6, 7, 8, 9, 11, 12]

La respuesta es correcta.  
Veamos un poco más en profundidad cómo funciona.  
¿Cómo encuentra la suma 7?
`array = [1,3,3,5]`  
Se necesita sumar 1+3+3, en el algoritmo existe un punto donde esto ocurrirá: `suma = sum(array[i:j+1])`. Esta instrucción sumará 1+3, luego, el for inferior sumará el otro 3.  
Si seguimos esa ejecución, el algoritmo también sumará 1+3+5 = 9 en el ciclo siguiente.

La complejidad del algoritmo es de $O(N^3)$. Y sinceramente no creo que el algoritmo funcione en todos los casos. Sin embargo, ya con una complejidad tan grande no vale la pena seguir buscando.

## Progamación dinámica

Debemos identificar el problema que se repite una y otra vez.  
Se puede decir que los resultados posibles abarcan de 0, no elegir elementos, a la suma de todos los elementos del array.

En nuestro ejemplo, todas las posiblidades van desde 0 hasta 12. 
Podemos declarar una f(x), donde x se mueve en este rango.

In [11]:
x = range(0,sum(array))

In [12]:
x

range(0, 12)

Ahora, f(x) arroja verdadero si x se puede obtener mediante la suma de los elementos del array: `array = [1,3,3,5]`, o falso si la suma no se puede obtener con los elementos del array.

Por ejemplo:
* f(0) = 1, ya que es posible obtener x con la suma de los elmentos. En este caso, no se suma ningún elemento.
* f(1) = 1, se puede obtenere seleccionando el primer elemento.
* f(2) = 0, ya que no se puede obtener x sumando nignún elemento del array.
* f(7) = 1, se obtiene sumando $1+3+3$.


Finalmente, la clave para reducir la complejidad es agregar un segundo parámetro que indique la ubicación dentro del array. $f(x,k)$ donde k indica en qué elemento del array estamos.

Entonces, la pregunta que hacemos es ¿se puede obtener este número sumando los elementos que hay en el array hasta k?

In [29]:
#array original
array = [1,3,3,5]

k_max = len(array)
x_max = sum(array)
f_xk = []
for i in range(x_max+1):
    f_xk.append([False]*(k_max+1))
#array que toma en cuenta no tomar ningún valor
array = [0] + array
#se puede obtener una suma de cero con cero elementos
f_xk[0][0] = True

for x in range(0,x_max+1):
    for k in range(1,k_max+1): 
        # se puede obtener esta suma con el valor anterior
        # o, si resto el valor actual, se obtiene esa suma con el valor anterior
        f_xk[x][k] = f_xk[x-array[k]][k-1] or f_xk[x][k-1] 

In [30]:
f_xk

[[True, True, True, True, True],
 [False, True, True, True, True],
 [False, False, False, False, False],
 [False, False, True, True, True],
 [False, False, True, True, True],
 [False, False, False, False, True],
 [False, False, False, True, True],
 [False, False, False, True, True],
 [False, False, False, False, True],
 [False, False, False, False, True],
 [False, False, False, False, False],
 [False, False, False, False, True],
 [False, False, False, False, True]]

En la matriz de arriba podemos ver que a partir del primer valor que dice True se puede obtener la suma correspondiente. Si vemos la fila 10, recordando que se enumeran de 0 a 12, la fila 10 está llena de False ya que no se puede obtener esa suma.  
Sin embargo, la fila 11 sí tiene un True al final, lo que significa que hay que tomar en cuenta todos los elementos, no sumar todos los elementos, sólo tenerlos en cuenta. Ya que si a 11 le restamos 5, se produce 6. Ese 6 se puede obtener sumando elementos anteriores y eso es lo que importa.

In [32]:
solucion = []
for i in range(len(f_xk)):
    if(True in f_xk[i]):
        solucion.append(i)

In [34]:
solucion

[0, 1, 3, 4, 5, 6, 7, 8, 9, 11, 12]

Se obtuvo la misma respuesta que en el algoritmo anterior, pero esta vez con una complejidad $O(N^2)$ y no cúbica.

In [51]:
#array original
array = [1,3,3,10,11]

k_max = len(array)
x_max = sum(array)
f_xk = []
for i in range(x_max+1):
    f_xk.append([False]*(k_max+1))
#array que toma en cuenta no tomar ningún valor
array = [0] + array
#se puede obtener una suma de cero con cero elementos
f_xk[0][0] = True

for x in range(0,x_max+1):
    for k in range(1,k_max+1): 
        # se puede obtener esta suma con el valor anterior
        # o, si resto el valor actual, se obtiene esa suma con el valor anterior
        f_xk[x][k] = f_xk[x-array[k]][k-1] or f_xk[x][k-1] 
        
solucion_pd = []
for i in range(len(f_xk)):
    if(True in f_xk[i]):
        solucion_pd.append(i)

In [52]:
solucion_pd

[0, 1, 3, 4, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 24, 25, 27, 28]

In [53]:
array = [1,3,3,10,11]
#debemos encontrar todas las sumas posibles
sumas_posibles = [0]*(sum(array)+1) #array de len = suma de todos los elementos+1
sumas_posibles[0] = 1 #no se elige ningún elemento

#tomamos el primer elemento
for i in range(len(array)):
    sumas_posibles[array[i]] = 1 #se elige un solo elemento
    for j in range(i+1,len(array)):
        # se suman dos elementos
        suma = array[i]+array[j]
        sumas_posibles[suma]=1
        
        # suman todos los elementos entre el primero y el segundo
        suma = sum(array[i:j+1])
        
        #la suma anterior se suma a otros números del array
        for k in range(j+1,len(array)):
            suma2= suma + array[k]
            sumas_posibles[suma2]=1
            
solucion_lenta = []
for i in range(len(sumas_posibles)):
    if(sumas_posibles[i]==1):
        solucion_lenta.append(i)

In [54]:
solucion_lenta

[0, 1, 3, 4, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 24, 27, 28]

En este momento acabo de ver la solución lenta y en este caso falla porque necesita sumar 1 10 11 para llegar a 22 y el algoritmo sólo toma rangos de izquierda a derecha. En este caso tendría que sumar 11 10 y llegar hasta el 1 en sentido inverso.  
Esto demuestra la complejidad del problema y lo poco efectiva que puede ser una respuesta de fuerza bruta.